In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from astropy.table import Table
import glob
from scipy.interpolate import RegularGridInterpolator

In [ ]:
# -------------------------------
# Step 0: Define global parameters and functions
# -------------------------------

tan = (1.7259 - 1.9644) / (149.66 - 150.31) 
theta = -np.arctan(tan)

def rotate_coordinates(ra, dec, theta):
    x = np.cos(theta) * ra + np.sin(theta) * dec
    y = -np.sin(theta) * ra + np.cos(theta) * dec
    return x, y

c0 = 3e5
physical_width = 35  # in (Mpc/h)
H_0 = 70
Omega_m = 0.3
Omega_l = 0.7

def slice_width(z):
    return physical_width * 100 / c0 * np.sqrt(Omega_m * (1+z)**3 + Omega_l)

def redshift_bins(zmin, zmax):
    centers = [zmin + 0.5 * slice_width(zmin)]
    i = 0
    while centers[i] + slice_width(centers[i]) < zmax:
        centers.append(centers[i] + slice_width(centers[i]))
        i += 1
    centers = np.array(centers)
    edges = np.zeros((len(centers), 2))
    for i in range(len(centers)):
        edges[i, 0] = centers[i] - slice_width(centers[i]) / 2
        edges[i, 1] = centers[i] + slice_width(centers[i]) / 2
    return centers, edges

# -------------------------------
# Step 1: Load Data and compute rotated positions
# -------------------------------
data_csv_path = "path to your data"
Data = pd.read_csv(data_csv_path)
Data['x'], Data['y'] = rotate_coordinates(Data['ra_detec'], Data['dec_detec'], theta)


x_min = Data['x'].min()
x_max = Data['x'].max()
y_min = Data['y'].min()
y_max = Data['y'].max()

# -------------------------------
# Step 2: Reconstruct the common grid
# -------------------------------
mesh_y = 120
width = x_max - x_min
height = y_max - y_min
wh_ratio = width / height
mesh_x = int(wh_ratio * mesh_y)

gr_x = np.linspace(x_min, x_max, mesh_x)
gr_y = np.linspace(y_min, y_max, mesh_y)

# -------------------------------
# Step 3: Get slice information and set up output matrix
# -------------------------------
z_min_val, z_max_val = 0.4, 9.5
slice_centers, _ = redshift_bins(z_min_val, z_max_val)
num_slices = len(slice_centers)
n_galaxies = len(Data)

density_excess_matrix = np.zeros((n_galaxies, num_slices))

# -------------------------------
# Step 4: Loop over slices: interpolate density excess
# -------------------------------
density_dir = r'~\Results\outputs\density'

# Build (y, x) points once — RegularGridInterpolator expects this order
interp_points = Data[['y', 'x']].to_numpy()

for s in range(num_slices):
    pattern = os.path.join(density_dir, f"output_slice_{s}_*.npz")
    files_found = glob.glob(pattern)

    if not files_found:
        print(f"Warning: No file found for slice {s}")
        continue
    
    slice_file = files_found[0]
    slice_data = np.load(slice_file)
    density_excess_grid = slice_data['density_excess']

    # Create interpolator and evaluate all galaxies at once
    interp = RegularGridInterpolator((gr_y, gr_x), density_excess_grid, bounds_error=False, fill_value=np.nan)
    density_excess_matrix[:, s] = interp(interp_points)

    print(f"Processed slice {s}: {os.path.basename(slice_file)}")


In [ ]:
def setup(work_path='.'):
    '''
    Set up all of the necessary directories
    '''
    for subdir in ('inputs', 'outputs', 'bin', 
                   'outputs/plots', 'outputs/weights', 'outputs/density'):
        path = os.path.join(work_path, subdir)
        if not os.path.exists(path):
            os.makedirs(path)
            print(f'Built directory: {os.path.abspath(path)}') 
    
    outputs_dir = os.path.join(work_path, 'outputs')
    plots_dir = os.path.join(work_path, 'outputs', 'plots')
    inputs_dir = os.path.join(work_path, 'inputs')
    weight_dir = os.path.join(work_path, 'outputs', 'weights')
    density_dir = os.path.join(work_path, 'outputs', 'density')
    return outputs_dir, plots_dir, inputs_dir, weight_dir, density_dir

cat_dir = "where you want to set up the catalog directories"

outputs_dir, plots_dir, inputs_dir, weights_dir, density_dir = setup(work_path=cat_dir)


threshold = 0.05

weights = np.load(os.path.join(weights_dir, f'weights_unthresholded_normalized_thresh{threshold}_lengh{physical_width}.npy'))
weights_block = np.load(os.path.join(weights_dir, f'weightsBlock_unthresholded_normalized_thresh{threshold}_lengh{physical_width}.npy'))
W = np.load(os.path.join(weights_dir, f'weightsBlock_thresh{threshold}_normalized_lengh{physical_width}.npy'))
normalized_delta_z_median = np.load(os.path.join(weights_dir, f'normalized_delta_z_median_thresh{threshold}_lengh{physical_width}.npy'))
delta_z_median = np.load(os.path.join(weights_dir, f'delta_z_median_thresh{threshold}_lengh{physical_width}.npy'))
count_in_zslice = np.load(os.path.join(weights_dir, f'count_in_zslice_thresh{threshold}_lengh{physical_width}.npy'))


In [ ]:
# Compute numerator: weighted sum along slices (axis 1)
weighted_sum = np.nansum(W * density_excess_matrix, axis=1)

# Compute denominator: sum of weights per galaxy
sum_weights = np.nansum(W, axis=1)

final_density = np.where(sum_weights != 0, weighted_sum / sum_weights, 0)

# Add the resulting final_density as a new column in Data
Data['density_excess'] = final_density


In [ ]:
# Save the Data table with the new density_excess column
output_csv_path = os.path.join(inputs_dir, 'COSMOSWeb_density_excess.csv')
Data.to_csv(output_csv_path, index=False)

In [ ]:
Data